# Cluster graph topology

In [ ]:
import re
import networkx as nx
from networkx.algorithms import isomorphism
import json

def graphShape(shapePath):
    with open(shapePath, 'r') as f:
        data = f.read()
    solveSpec = json.loads(data)
    G = nx.Graph()
    for i, _, j, _ in solveSpec['bindings']:
        G.add_edge(i, j)
    return G

def graphsFromClusters(line):
    clusterGraphs = []
    clusters = re.finditer('\[.+?\]', line)

    for cluster in clusters:
        G = nx.Graph()
        matches = re.finditer(
            '(\d+) -> \(((?:\d+ ?)+)\)', cluster.group()
        )
        for m in matches:
            source = m.group(1)
            for dest in m.group(2).split(' '):
                if int(source) == int(dest):
                    print("AAAAAAAAAAAAaaaaaaaaaaaaaaaaaaaa")
                G.add_edge(int(source), int(dest))
        clusterGraphs.append(G)
    return clusterGraphs

def getGraphOverlap(g1, g2, cutoff=1):
    sizeFrac = len(g2)/len(g1)
    return sizeFrac if sizeFrac <= 1 and sizeFrac >= cutoff and isomorphism.GraphMatcher(
        nx.line_graph(g1), nx.line_graph(g2)
    ).subgraph_is_isomorphic() else 0

def getClusterYield(line, refGraph, cutoff):
    return sum(getGraphOverlap(refGraph, g, cutoff) for g in graphsFromClusters(line))

def readClusters(clustersPath, shapePath, cutoff):
    refGraph = graphShape(shapePath)
    clusters = []
    with open(clustersPath) as f:
        for line in f:
            y = getClusterYield(line, refGraph, cutoff)
            print(y, end=' ')
            clusters.append(y)
    return clusters

In [ ]:
import numpy as np
def coordsFromConf(confPath):
    coords = []
    box = []
    with open(confPath, 'r') as f:
        for line in f:
            vals = line.split()
            if (len(vals) == 5 and vals[0] == 'b'):
                box = [float(v) for v in vals[-3:]]
            if (len(vals) == 15):
                coords.append([float(v)%box[i] for i,v in enumerate(vals[:3])])
    return {i: np.array(c) for i, c in enumerate(coords)}

In [ ]:
line = '11 ( 0 2 3 4 4 4 4 )  [7 -> (72 79 62 80 31), 29 -> (31)]( 0 1 2 3 4 4 4 4 4 )  [2 -> (88 85 49 77 33), 10 -> (22 45), 22 -> (33)]( 0 1 2 3 4 4 4 4 4 )  [0 -> (57 35 43 74 66), 11 -> (41 26), 26 -> (35)]( 0 1 2 3 4 4 4 4 4 )  [3 -> (87 34 55 58 59), 12 -> (28 68), 28 -> (34)]( 1 4 )  [13 -> (67)]( 0 1 2 3 4 4 4 4 4 )  [4 -> (51 30 40 65 50), 14 -> (48 27), 27 -> (30)]( 0 1 2 3 4 4 4 4 4 )  [9 -> (56 71 82 32 60), 15 -> (73 23), 23 -> (32)]( 0 1 2 3 4 4 4 4 4 )  [8 -> (47 63 46 37 86), 16 -> (25 52), 25 -> (37)]( 0 1 2 3 4 4 4 4 4 )  [5 -> (89 70 39 84 78), 17 -> (24 54), 24 -> (39)]( 0 1 2 3 4 4 4 4 4 )  [6 -> (75 69 36 64 53), 18 -> (20 76), 20 -> (36)]( 0 1 2 3 4 4 4 4 4 )  [1 -> (42 44 61 81 38), 19 -> (21 83), 21 -> (38)]'
swanPath = '../shapes/swan.json'
getClusterYield(
    line,
    graphShape(swanPath),
    0.9
)

In [ ]:
nx.draw(graphShape('../shapes/swan.json'))

In [ ]:
import matplotlib.pyplot as plt
for i, G in enumerate(graphsFromClusters(line)):
    plt.figure(i+1)
    nx.draw(G)

In [ ]:
nx.draw(graphShape('../shapes/filled_cube.json'))

In [ ]:
getClusterYield(
    '11 ( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 19 20 21 24 25 26 )  [0 -> (10 20 32), 10 -> (56 40 69), 20 -> (56 100), 32 -> (69 100 114), 40 -> (77 82), 56 -> (77 126), 69 -> (126 82 138), 77 -> (197 209), 82 -> (99), 99 -> (144 138), 100 -> (216 126), 114 -> (138), 126 -> (209 264 153), 138 -> (153), 144 -> (209 241 153), 197 -> (255), 209 -> (255), 216 -> (264), 241 -> (255), 255 -> (264)]( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 16 17 18 19 20 21 23 24 25 26 )  [1 -> (23 36 17), 17 -> (54 44 65), 23 -> (179 54 108), 36 -> (108 65 113), 44 -> (70 84), 54 -> (189 124 70), 65 -> (124 84 130), 70 -> (194 202), 84 -> (91), 91 -> (147 130), 108 -> (215 124 169), 113 -> (169 130), 124 -> (265 202), 147 -> (202 248), 179 -> (215 189), 189 -> (265), 194 -> (253), 202 -> (253), 215 -> (265), 231 -> (265 248), 248 -> (253), 253 -> (265)]( 0 1 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 )  [2 -> (13 37), 13 -> (50 41 61), 37 -> (102 61 110), 41 -> (75 86), 50 -> (75 180 120), 61 -> (120 131 86), 75 -> (190 207), 86 -> (96), 96 -> (140 131), 102 -> (214 120 161), 110 -> (131 161), 120 -> (207 159), 131 -> (159), 140 -> (243 207 159), 159 -> (233 161), 161 -> (225), 172 -> (214 180), 190 -> (257), 207 -> (257), 214 -> (225), 225 -> (233), 233 -> (243), 243 -> (257)]( 0 1 2 3 4 5 6 7 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 )  [3 -> (16 27 35), 16 -> (67 43 59), 27 -> (103 59 177), 35 -> (67 116 103), 43 -> (72), 59 -> (183 72 122), 67 -> (137 122), 72 -> (204 195), 92 -> (137 146), 103 -> (162 122 219), 116 -> (137 162), 122 -> (204 157), 137 -> (157), 146 -> (204 157 245), 157 -> (162 237), 162 -> (229), 177 -> (183 219), 195 -> (250), 204 -> (250), 219 -> (229), 229 -> (237), 237 -> (245), 245 -> (250)]( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 16 17 18 19 20 21 22 25 26 )  [4 -> (12 39 25), 12 -> (47 60 52), 25 -> (101 170 52), 39 -> (60 118 101), 47 -> (88 71), 52 -> (71 125 186), 60 -> (136 88 125), 71 -> (192 206), 88 -> (93), 93 -> (136), 101 -> (125 164 211), 118 -> (136 164), 125 -> (261 206), 164 -> (223), 170 -> (186 211), 186 -> (261), 192 -> (256), 206 -> (256), 211 -> (261 223), 256 -> (261)]( 0 1 2 3 4 6 7 8 9 10 11 13 14 15 16 17 18 19 20 21 22 23 24 25 26 )  [5 -> (34 21 18), 18 -> (66 49), 21 -> (173 105), 34 -> (105 66 115), 49 -> (79 80), 66 -> (132 80), 79 -> (196 203), 80 -> (94), 94 -> (148 132), 105 -> (210 168), 115 -> (168 132), 132 -> (150), 148 -> (244 150 203), 150 -> (230 168), 168 -> (228), 173 -> (210 181), 181 -> (267), 196 -> (251), 203 -> (251), 210 -> (228 267), 228 -> (230), 230 -> (267 244), 244 -> (251), 251 -> (267)]( 0 1 2 3 4 5 6 7 8 9 11 12 13 14 15 16 17 18 19 20 22 23 24 25 26 )  [6 -> (19 24 30), 19 -> (58 48 68), 24 -> (58 171), 30 -> (68 111), 48 -> (78 83), 58 -> (78 188 128), 68 -> (83 128 133), 78 -> (198 208), 83 -> (95), 95 -> (142 133), 111 -> (160 133), 128 -> (208 263 152), 133 -> (152), 142 -> (208 242 152), 152 -> (239 160), 160 -> (221), 171 -> (188), 188 -> (263), 198 -> (258), 208 -> (258), 221 -> (239), 239 -> (242 263), 242 -> (258), 258 -> (263)]( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 )  [7 -> (11 29 33), 11 -> (57 46 63), 29 -> (57 175 106), 33 -> (106 63 117), 46 -> (74 87), 57 -> (187 74 123), 63 -> (87 123 135), 74 -> (193 201), 87 -> (98), 98 -> (149 135), 106 -> (217 123 165), 117 -> (135 165), 123 -> (201 268 154), 135 -> (154), 149 -> (201 240 154), 154 -> (236 165), 165 -> (220), 175 -> (187 217), 187 -> (268), 193 -> (254), 201 -> (254), 217 -> (268 220), 220 -> (236), 236 -> (268 240), 240 -> (254), 254 -> (268)]( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 21 22 23 24 26 )  [8 -> (15 38 28), 15 -> (45 62 51), 28 -> (51 178 104), 38 -> (112 62 104), 45 -> (89 73), 51 -> (73 121 185), 62 -> (89 139 121), 89 -> (97), 97 -> (139 143), 104 -> (166 121 218), 112 -> (139 166), 121 -> (158 269), 139 -> (158), 143 -> (158 246), 158 -> (166 232), 166 -> (226), 178 -> (218 185), 185 -> (269), 218 -> (269 226), 226 -> (232), 232 -> (269 246)]( 2 5 12 15 16 17 18 21 22 23 26 )  [26 -> (176 55), 55 -> (184 127), 127 -> (266 155), 155 -> (238 163), 163 -> (227), 176 -> (184 213), 184 -> (266), 213 -> (266 227), 227 -> (238), 238 -> (266)]( 0 1 2 3 4 5 6 7 8 9 10 10 11 12 13 14 14 15 15 16 17 18 19 20 20 21 22 22 23 23 24 24 25 25 26 26 )  [9 -> (31 14 22), 14 -> (64 42 53), 22 -> (107 53 174), 31 -> (119 64 109), 42 -> (85 76), 53 -> (76 182), 64 -> (134 129 85), 76 -> (200 191), 85 -> (90), 90 -> (134 145), 107 -> (212), 109 -> (167 129), 119 -> (134 167), 129 -> (260 151 205), 134 -> (151), 141 -> (156 200 247), 145 -> (249 151 205), 151 -> (167 235), 156 -> (234), 167 -> (222), 174 -> (212 182), 182 -> (262), 191 -> (252), 200 -> (252), 205 -> (259), 212 -> (224 262), 222 -> (235), 224 -> (234), 234 -> (262 247), 235 -> (260 249), 247 -> (252), 249 -> (259), 252 -> (262), 259 -> (260)]', 
    graphShape('../shapes/filled_cube.json'),
    0.75
)

In [ ]:
[getGraphOverlap(graphShape('../shapes/filled_cube.json'), g, 0.75) for g in graphsFromClusters(
    '11 ( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 19 20 21 24 25 26 )  [0 -> (10 20 32), 10 -> (56 40 69), 20 -> (56 100), 32 -> (69 100 114), 40 -> (77 82), 56 -> (77 126), 69 -> (126 82 138), 77 -> (197 209), 82 -> (99), 99 -> (144 138), 100 -> (216 126), 114 -> (138), 126 -> (209 264 153), 138 -> (153), 144 -> (209 241 153), 197 -> (255), 209 -> (255), 216 -> (264), 241 -> (255), 255 -> (264)]( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 16 17 18 19 20 21 23 24 25 26 )  [1 -> (23 36 17), 17 -> (54 44 65), 23 -> (179 54 108), 36 -> (108 65 113), 44 -> (70 84), 54 -> (189 124 70), 65 -> (124 84 130), 70 -> (194 202), 84 -> (91), 91 -> (147 130), 108 -> (215 124 169), 113 -> (169 130), 124 -> (265 202), 147 -> (202 248), 179 -> (215 189), 189 -> (265), 194 -> (253), 202 -> (253), 215 -> (265), 231 -> (265 248), 248 -> (253), 253 -> (265)]( 0 1 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 )  [2 -> (13 37), 13 -> (50 41 61), 37 -> (102 61 110), 41 -> (75 86), 50 -> (75 180 120), 61 -> (120 131 86), 75 -> (190 207), 86 -> (96), 96 -> (140 131), 102 -> (214 120 161), 110 -> (131 161), 120 -> (207 159), 131 -> (159), 140 -> (243 207 159), 159 -> (233 161), 161 -> (225), 172 -> (214 180), 190 -> (257), 207 -> (257), 214 -> (225), 225 -> (233), 233 -> (243), 243 -> (257)]( 0 1 2 3 4 5 6 7 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 )  [3 -> (16 27 35), 16 -> (67 43 59), 27 -> (103 59 177), 35 -> (67 116 103), 43 -> (72), 59 -> (183 72 122), 67 -> (137 122), 72 -> (204 195), 92 -> (137 146), 103 -> (162 122 219), 116 -> (137 162), 122 -> (204 157), 137 -> (157), 146 -> (204 157 245), 157 -> (162 237), 162 -> (229), 177 -> (183 219), 195 -> (250), 204 -> (250), 219 -> (229), 229 -> (237), 237 -> (245), 245 -> (250)]( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 16 17 18 19 20 21 22 25 26 )  [4 -> (12 39 25), 12 -> (47 60 52), 25 -> (101 170 52), 39 -> (60 118 101), 47 -> (88 71), 52 -> (71 125 186), 60 -> (136 88 125), 71 -> (192 206), 88 -> (93), 93 -> (136), 101 -> (125 164 211), 118 -> (136 164), 125 -> (261 206), 164 -> (223), 170 -> (186 211), 186 -> (261), 192 -> (256), 206 -> (256), 211 -> (261 223), 256 -> (261)]( 0 1 2 3 4 6 7 8 9 10 11 13 14 15 16 17 18 19 20 21 22 23 24 25 26 )  [5 -> (34 21 18), 18 -> (66 49), 21 -> (173 105), 34 -> (105 66 115), 49 -> (79 80), 66 -> (132 80), 79 -> (196 203), 80 -> (94), 94 -> (148 132), 105 -> (210 168), 115 -> (168 132), 132 -> (150), 148 -> (244 150 203), 150 -> (230 168), 168 -> (228), 173 -> (210 181), 181 -> (267), 196 -> (251), 203 -> (251), 210 -> (228 267), 228 -> (230), 230 -> (267 244), 244 -> (251), 251 -> (267)]( 0 1 2 3 4 5 6 7 8 9 11 12 13 14 15 16 17 18 19 20 22 23 24 25 26 )  [6 -> (19 24 30), 19 -> (58 48 68), 24 -> (58 171), 30 -> (68 111), 48 -> (78 83), 58 -> (78 188 128), 68 -> (83 128 133), 78 -> (198 208), 83 -> (95), 95 -> (142 133), 111 -> (160 133), 128 -> (208 263 152), 133 -> (152), 142 -> (208 242 152), 152 -> (239 160), 160 -> (221), 171 -> (188), 188 -> (263), 198 -> (258), 208 -> (258), 221 -> (239), 239 -> (242 263), 242 -> (258), 258 -> (263)]( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 )  [7 -> (11 29 33), 11 -> (57 46 63), 29 -> (57 175 106), 33 -> (106 63 117), 46 -> (74 87), 57 -> (187 74 123), 63 -> (87 123 135), 74 -> (193 201), 87 -> (98), 98 -> (149 135), 106 -> (217 123 165), 117 -> (135 165), 123 -> (201 268 154), 135 -> (154), 149 -> (201 240 154), 154 -> (236 165), 165 -> (220), 175 -> (187 217), 187 -> (268), 193 -> (254), 201 -> (254), 217 -> (268 220), 220 -> (236), 236 -> (268 240), 240 -> (254), 254 -> (268)]( 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 21 22 23 24 26 )  [8 -> (15 38 28), 15 -> (45 62 51), 28 -> (51 178 104), 38 -> (112 62 104), 45 -> (89 73), 51 -> (73 121 185), 62 -> (89 139 121), 89 -> (97), 97 -> (139 143), 104 -> (166 121 218), 112 -> (139 166), 121 -> (158 269), 139 -> (158), 143 -> (158 246), 158 -> (166 232), 166 -> (226), 178 -> (218 185), 185 -> (269), 218 -> (269 226), 226 -> (232), 232 -> (269 246)]( 2 5 12 15 16 17 18 21 22 23 26 )  [26 -> (176 55), 55 -> (184 127), 127 -> (266 155), 155 -> (238 163), 163 -> (227), 176 -> (184 213), 184 -> (266), 213 -> (266 227), 227 -> (238), 238 -> (266)]( 0 1 2 3 4 5 6 7 8 9 10 10 11 12 13 14 14 15 15 16 17 18 19 20 20 21 22 22 23 23 24 24 25 25 26 26 )  [9 -> (31 14 22), 14 -> (64 42 53), 22 -> (107 53 174), 31 -> (119 64 109), 42 -> (85 76), 53 -> (76 182), 64 -> (134 129 85), 76 -> (200 191), 85 -> (90), 90 -> (134 145), 107 -> (212), 109 -> (167 129), 119 -> (134 167), 129 -> (260 151 205), 134 -> (151), 141 -> (156 200 247), 145 -> (249 151 205), 151 -> (167 235), 156 -> (234), 167 -> (222), 174 -> (212 182), 182 -> (262), 191 -> (252), 200 -> (252), 205 -> (259), 212 -> (224 262), 222 -> (235), 224 -> (234), 234 -> (262 247), 235 -> (260 249), 247 -> (252), 249 -> (259), 252 -> (262), 259 -> (260)]'
)]

In [ ]:
graphShape('../shapes/filled_cube.json').nodes

In [ ]:
def graphFromLastestClusters(path):
    with open(path, 'r') as f:
        for line in f:
            pass
        last_line = line
    return graphsFromClusters(last_line)

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def _format_axes(ax):
    """Visualization options for the 3D axes."""
    # Turn gridlines off
    ax.grid(False)
    # Suppress tick labels
    for dim in (ax.xaxis, ax.yaxis, ax.zaxis):
        dim.set_ticks([])
    # Set axes labels
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    
def drawWithCoords(G, coords, i=0):
    pos = nx.spring_layout(G, dim=3, weight=1000, iterations=1000)
    # Extract node and edge positions from the layout
    node_xyz = np.array([coords[v] for v in sorted(G)])
    print([v for v in G])
    edge_xyz = np.array([(coords[u], coords[v]) for u, v in G.edges()])
    
    fig = plt.figure(i+1)
    ax = fig.add_subplot(111, projection="3d")
    
    # Plot the nodes - alpha is scaled by "depth" automatically
    ax.scatter(*node_xyz.T, s=100, ec="w")

    # Plot the edges
    for vizedge in edge_xyz:
        ax.plot(*vizedge.T, color="tab:gray")

        _format_axes(ax)
    fig.tight_layout()
    plt.show()
    
path = 'new_0.1/solid_full/nt0/T_0.06/'

filled_cube = graphShape('../shapes/filled_cube.json')
coords = coordsFromConf(path+'last_conf.dat')
for i, G in enumerate(graphFromLastestClusters(path+'clusters.txt')):
    print(
        #isomorphism.GraphMatcher(
        #    nx.line_graph(filled_cube), nx.line_graph(G)
        #).subgraph_is_isomorphic(),
        getGraphOverlap(filled_cube, G, 0),
         " - {} of {} cubes".format(len(G), len(filled_cube)),
          G.nodes
     )
    drawWithCoords(G, coords, i)



In [ ]:
F = graphFromLastestClusters(path+'clusters.txt')[1]

In [ ]:
drawWithCoords(F, coords)

In [ ]:
isomorphism.GraphMatcher(
    filled_cube, F
).subgraph_is_isomorphic()

In [ ]:
isomorphism.GraphMatcher(
    nx.line_graph(filled_cube), nx.line_graph(F)
).subgraph_is_isomorphic()

In [ ]:
nx.draw(nx.line_graph(F))

In [ ]:
#readClusters('new_0.1/solid_full/nt0/T_0.06/clusters.txt', '../shapes/filled_cube.json', 0.75)

# Check if you have isomorphism while removing nodes

In [ ]:
import random
G = graphShape('../shapes/filled_cube.json')
nx.draw(G)

In [ ]:
H = G.copy()
while len(H) > 0:
    H.remove_node(random.choice([n for n in H.nodes]))
    #plt.figure(len(G)-len(H)+1)
    #nx.draw(H)
    print(isomorphism.GraphMatcher(G, H).subgraph_is_isomorphic())

# Check if you have isomorphism while removing edges

In [ ]:
H = G.copy()
while len(H) > len(G)/2:
    H.remove_edge(*random.choice([n for n in H.edges]))
    print(isomorphism.GraphMatcher(G, H).subgraph_is_isomorphic())

In [ ]:
H = G.copy()
while len(H) > len(G)/2:
    H.remove_edge(*random.choice([n for n in H.edges]))
    print(isomorphism.GraphMatcher(nx.line_graph(G), nx.line_graph(H)).subgraph_is_isomorphic())

# Minimal filled cube

In [ ]:
path = 'new_0.1/solid/nt0/T_0.01/'

filled_cube = graphShape('../shapes/filled_cube.json')
coords = coordsFromConf(path+'last_conf.dat')
for i, G in enumerate(graphFromLastestClusters(path+'clusters.txt')):
    print(
        getGraphOverlap(filled_cube, G, 0),
        " - {} of {} cubes".format(len(G), len(filled_cube)),
         G.nodes
    )
    drawWithCoords(G, coords, i)
    